# Parsing

## CSV Parsing

In [10]:
# importing the Python csv library
import csv

In [19]:
with open("files/switch_record.csv", 'r') as csv_file:
    content = csv.reader(csv_file, dialect='excel')
    for a in content:
        print a

IOError: [Errno 2] No such file or directory: 'files/switch_record.xlsx'

## JSON Parsing

In [ ]:
# importing the Python json library
import json

### Importing JSON to Dict { }

**From string**

In [ ]:
# Example string
json_string = '{"first_name": "Guido", "last_name":"Rossum"}'
json_string

In [ ]:
parsed_json = json.loads(json_string)
parsed_json

**From file**

In [ ]:
# %load files/example.json

In [ ]:
with open('files/example.json', 'r') as f:
    parsed_json = json.load(f)
    
parsed_json

### Exporting Dict { } to JSON

In [ ]:
# Example dict
d = { 'first_name': 'Guido',
      'second_name': 'Rossum',
      'titles': ['BDFL', 'Developer'],
    }
d

In [ ]:
json_string = json.dumps(d)
json_string

## XML Parsing

In [ ]:
import xml.etree.ElementTree as ET

### Importing XML to ElementTree

**From string**

In [ ]:
# Example string
xml_string = """<catalog>
   <book id="bk101">
      <author>Gambardella, Matthew</author>
      <title>XML Developer's Guide</title>
      <genre>Computer</genre>
   </book>
</catalog>"""
xml_string

In [ ]:
root = ET.fromstring(xml_string)
print root.tag
print root.attrib

**From file**

In [ ]:
# %load files/example.xml

In [ ]:
tree = ET.parse('files/example.xml')
root = tree.getroot()
print root.tag
print root.attrib

### Exporting ElementTree to XML

In [ ]:
tree.write("files/example_output.xml")

In [ ]:
# %load files/example_output.xml

### Getting items

In [ ]:
# Each XML element contains a tag, an attribute (optional), a text and a list of childs
# XML Element: <tag attr:attr_value> text </tag>

print root.tag            # tag    : name of XML element
print root.attrib         # attrib : attribute of XML element
print root.text           # text   : content of XML element
for child in root: # loop through all subchildrens
    print "\t", child.tag, child.attrib, child.text
    for subchild in child:
        print "\t\t", subchild.tag, subchild.attrib, subchild.text
    print
    
children = list(root)     #Get children

print children
print root

In [ ]:
# iter('element_tag') - search all subtrees
for title in root.iter('title'): 
    print title.text

In [ ]:
# findall('element_tag') - get direct childs of parent
for book in root.findall('book'):
    # Get items
    book_id = book.get('id')          # Get attribute 'id' of 'book' element
    title = book.find('title')        # Get first child named 'title'
    author = book.find('author')
    genre = book.find('genre')
    price = book.find('price')
    pdate = book.find('publish_date')
    descr = book.find('description')
    
    # Print items
    print "ID ", book_id
    print title.tag, title.text
    print author.tag, author.text
    print genre.tag, genre.text
    print price.tag, price.text
    print pdate.tag, pdate.text
    print descr.tag, descr.text
    print

In [ ]:
# find('element_tag') - finds first child with tag
for book_content in root.find('book'):
    print book_content

### Modifying items

In [ ]:
# set('attribute', 'attribute_value')
for price in root.iter('price'):
    price.text = str(float(price.text) + 1)
    price.set('increased', 'yes')
    print price.text, price.attrib

In [ ]:
# Create new element
new_book = ET.Element('book', {'id': 'bk113'})
new_book.text = "\n"
                      
# Create sub elements
author = ET.SubElement(new_book, 'author')
title = ET.SubElement(new_book, 'title')
genre = ET.SubElement(new_book, 'genre')
price = ET.SubElement(new_book, 'price')
pdate = ET.SubElement(new_book, 'publish_date')
descr = ET.SubElement(new_book, 'description')

# Populate sub elements
author.text = "J.K Rowlings"
title.text = "Harry Potter and the Sorcerer's Stone"
genre.text = "Fantasy"
price.text = "31.50"
pdate.text = "2001-10-16"
descr.text = "A very nice fantasy book."

# Add element to existing tree
root.append(new_book)

# ET.dump(new_book)

In [ ]:
# A ten times nicer way of doing it (focus on what matters)

def create_new_ET(element_name, attributes={}, elements={}):
    book = ET.Element('book', attributes)
    for key in elements:
        new_elem = ET.SubElement(book, key)
        new_elem.text = elements[key]
    return book

# Create new element
name     = 'book'
attr     = {'id': "bk114"}
elements = {'author': "J.K Rowlings",
            'title': "Harry Potter and the Chamber of Secrets",
            'genre': "Fantasy",
            'price': "35.50",
            'publish_date': "2002-10-15",
            'description': "The second volume of a very nice fantasy book."}
new_book = create_new_ET(name, attr, elements)

# Append to existing tree
root.append(new_book)

# ET.dump(root)

## Bonus: Pretty Print and Conversion

### Pretty Print

In [ ]:
def indent(elem, level=0):
    i = "\n" + level*"  "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "  "
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            indent(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i
            
indent(root)
tree.write("files/example_output.xml")
ET.dump(tree)

In [ ]:
# %load files/example_output.xml

### Etree to Dict { }

In [ ]:
from collections import defaultdict

def etree_to_dict(t):
    d = {t.tag: {} if t.attrib else None}
    children = list(t)
    if children:
        dd = defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.iteritems():
                dd[k].append(v)
        d = {t.tag: {k:v[0] if len(v) == 1 else v for k, v in dd.iteritems()}}
    if t.attrib:
        d[t.tag].update(('@' + k, v) for k, v in t.attrib.iteritems())
    if t.text:
        text = t.text.strip()
        if children or t.attrib:
            if text:
              d[t.tag]['#text'] = text
        else:
            d[t.tag] = text
    return d

In [ ]:
d = etree_to_dict(root)
d

### Dict { } to Etree

In [ ]:
def dict_to_etree(d):
    def _to_etree(d, root):
        if not d:
            pass
        elif isinstance(d, basestring):
            root.text = d
        elif isinstance(d, dict):
            for k,v in d.items():
                assert isinstance(k, basestring)
                if k.startswith('#'):
                    assert k == '#text' and isinstance(v, basestring)
                    root.text = v
                elif k.startswith('@'):
                    assert isinstance(v, basestring)
                    root.set(k[1:], v)
                elif isinstance(v, list):
                    for e in v:
                        _to_etree(e, ET.SubElement(root, k))
                else:
                    _to_etree(v, ET.SubElement(root, k))
        else: assert d == 'invalid type', (type(d), d)
    assert isinstance(d, dict) and len(d) == 1
    tag, body = next(iter(d.items()))
    node = ET.Element(tag)
    _to_etree(body, node)
    return node

In [ ]:
t = dict_to_etree(d)
indent(t)
ET.dump(t)

## Exercise: Analyze XML file
### Problem
**Objectives:** 
- Extract Maven plugins information from ```base-corporate-pom```.
- Add a new Maven plugin to ```base-corporate-pom```.
- Write Maven plugins information to a new file ```files/pom_maven_plugins.xml```

**Information:**

- ```base-corporate-pom``` pom.xml is located in ```files/pom.xml```


- XML plugin structure:
```
<project>
    <!-- Plugin version -->
    <properties>
        <maven-surefire-plugin.version>2.12.4</maven-surefire-plugin.version>
    </properties>
    <!-- Plugin info -->
    <build>
        <pluginManagement>
            <plugins>
                <plugin>
                    <groupId>org.apache.maven.plugin</groupId>
                    <artifactId>maven-surefire-plugin</version>
                    <version>$(maven-surefire-plugin.version)</version>
                 </plugin>
            </plugins>
         </pluginManagement>
    </build>
</project>
```
- Some plugins don't have a ```<version>``` or a ```<groupId>``` tag. Print "N/A" when there is no tag.
- All plugins have an ```<artifactId>``` tag.

**Desired output:**

maven-surefire

    groupId: org.apache.maven.plugins
    artifactId: maven-surefire-plugin
    version: 2.12.4

### Solution 1

In [ ]:
%load solutions/xml_solution.py

### Solution 2 (improved)

In [ ]:
%load solutions/xml_solution2.py

### Solution 3 (OOP Approach)

In [ ]:
%load solutions/xml_solution3.py

# REST API (Django)

In [ ]:
import oauth2 as oauth

class TwitterTest():
    
    CONSUMER_KEY = 'OdxPJBCh8YCON6MPKxsazlsCZ'
    CONSUMER_SECRET = 'shDDhOgg19ndqrEmqQ0VWrVEoi9GaOC1jrv34fLYER0241LQZx'
    ACCESS_TOKEN = '4457260453-xmLWkD6O6UXkandVfBKQnm0E4s18HQZwlyIi7qF'
    ACCESS_TOKEN_SECRET = 'lj89WiQLJfzPM8wNasPuzMBFeauIVxfmIRzEUeuGPIwmi'
    
    def testApi(self):
        consumer = oauth.Consumer(key=self.CONSUMER_KEY, secret=self.CONSUMER_SECRET)
        access_token = oauth.Token(key=self.ACCESS_TOKEN, secret=self.ACCESS_TOKEN_SECRET)
        client = oauth.Client(consumer, access_token)
        
        req_endpoint = "https://api.twitter.com/1.1/search/tweets.json?q=austin" 
        response, data = client.request(req_endpoint, method='GET')
        print response
        print data


''' MAIN '''
if __name__ == '__main__':
    tw = TwitterTest()
    tw.testApi()
    
    print “DONE” 

# SQL Data Access (MySQL)

In [2]:
import mysql.connector

## Connecting to MySQL DB

In [3]:
cnx = mysql.connector.connect(user='root', password='password',
                              host='localhost')
cnx.close()
cnx

**Lots of connection arguments**

In [4]:
config = {
  'user': 'root',
  'password': 'password',
  'host': 'localhost',
  'database': 'tutorial',
  'raise_on_warnings': True,
}

cnx = mysql.connector.connect(**config)
cnx.close()
cnx

ProgrammingError: 1049 (42000): Unknown database 'tutorial'

**Handling connection errors**

In [5]:
from mysql.connector import errorcode

config = {
  'user': 'root',
  'password': 'wrong_password',
  'host': 'localhost',
  'raise_on_warnings': True,
}

# Exception handling code
try:
    cnx = mysql.connector.connect(**config)
    
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print "Something is wrong with you user name or password"
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print "Database does not exist."
    else:
        print err
else:
    cnx.close()

Something is wrong with you user name or password


**Executing SQL statements**

***Note: All DDL (Data Definition Language) statements are executed using a handle structure known as 'cursor'.***

In [8]:
# A basic example for executing SQL queries

from mysql.connector import MySQLConnection

config = {
  'user': 'root',
  'password': 'password',
  'host': 'localhost',
  'raise_on_warnings': True,
}

def exec_query(cursor, query, query_args=None):
    try:
        if query_args is not None:
            query = query % query_args
            print "[INFO]  Executing SQL: \"%s\"..." % query,
            cursor.execute(query, query_args)
        else:
            print "[INFO]  Executing SQL: \"%s\"..." % query,
            cursor.execute(query)
        print " SUCCESS"
    except mysql.connector.Error as err:
        print
        print "[ERROR] " + err.msg
        exit(1)
        
if __name__ == '__main__':
    # SQL statements
    query_1 = "CREATE DATABASE test"
    query_2 = "CREATE TABLE people (first_name VARCHAR(25) PRIMARY KEY, last_name VARCHAR(25))"
    query_3 = "INSERT people(first_name, last_name) VALUES (%s, %s)"
    query_3_args = ("Olivier", "Cervello")
    query_4 = "SELECT * FROM people"
    query_4 = "DROP TABLE people"
    query_5 = "DROP DATABASE test"
    
    #connect to MySQL server
    cnx = MySQLConnection(**config)
    cursor = cnx.cursor()
    
    #create database 'test'
    exec_query(cursor, query_1)
    
    # connect to database 'test'
    config['database'] = 'test'
    cnx.connect(**config)
    
    # create table 'people' and insert a value
    exec_query(cursor, query_2)
    exec_query(cursor, query_3, query_3_args)
    
    # Commit changes (query_3)
    cnx.commit()
    
    # delete databases
    exec_query(cursor, query_4)
    exec_query(cursor, query_5)

    # close connection
    cnx.close()

[INFO]  Executing SQL: "CREATE DATABASE test"...  SUCCESS
[INFO]  Executing SQL: "CREATE TABLE people (first_name VARCHAR(25) PRIMARY KEY, last_name VARCHAR(25))"...  SUCCESS
[INFO]  Executing SQL: "INSERT people(first_name, last_name) VALUES (Olivier, Cervello)"...
[ERROR] Not all parameters were used in the SQL statement
[INFO]  Executing SQL: "DROP TABLE people"...  SUCCESS
[INFO]  Executing SQL: "DROP DATABASE test"...  SUCCESS


## Creating / Deleting a database

In [ ]:
from mysql.connector import MySQLConnection

config = {
  'user': 'root',
  'password': 'password',
  'host': 'localhost',
  'raise_on_warnings': True,
}

def create_database(cursor, db_name):
    query = "CREATE DATABASE %s" % db_name
    exec_query(cursor, query)
        
def get_database(cnx, db_name):
    cursor = cnx.cursor()
    print "[INFO]  Fetching database %s ..." % db_name
    try:
        cnx.database = db_name    
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_BAD_DB_ERROR:
            print "[ERROR] Database %s does not exist! Creating database..." % db_name
            create_database(cursor, db_name)
            cnx.database = db_name
        else:
            print(err)
            exit(1)
    print "[INFO]  Database %s selected." % db_name

def delete_database(cursor, db_name):
    query = "DROP DATABASE %s" % db_name
    exec_query(cursor, query)

if __name__ == '__main__':
    cnx = MySQLConnection(**config)
    cursor = cnx.cursor()
    get_database(cnx, 'new_db')
    delete_database(cursor, 'new_db')

## Creating / Deleting a table

In [ ]:
# create_table(cursor, table_name, table_ddl)
def create_table(cursor, table_name, args):
    args_str = ','.join(args)
    query = "CREATE TABLE %s" % table_name + "(" + args_str + ")"
    exec_query(cursor, query)
    
# delete_table(cursor, table_name, table_ddl)
def delete_table(cursor, table_name):
    query = "DROP TABLE %s" % table_name
    exec_query(cursor, query)
    
if __name__ == '__main__':
    cnx = MySQLConnection(user='root', password='password')
    cursor = cnx.cursor()
    get_database(cnx, 'new_db')
    
    # New table
    name = "people"
    args = ("emp_no int(11) NOT NULL",
            "first_name varchar(14) NOT NULL",
            "last_name varchar(14) NOT NULL",
            "PRIMARY KEY (emp_no)"
           )
    create_table(cursor, name, args)
#     delete_table(cursor, name)

## Populating a table

In [ ]:
# add_employee(cursor, args)
def add_employee(cursor, db_name, emp_no, first_name, last_name):
    query = "INSERT INTO people(emp_no,first_name, last_name) " \
            "VALUES(%s,%s,%s)"
    args = (emp_no, first_name, last_name)
    exec_query(cursor, query, args)
    
if __name__ == '__main__':
    # Connect to database
    cnx = MySQLConnection(**config)
    cursor = cnx.cursor()

    # Get database
    get_database(cnx, 'new_db')

    # Insert new employee
    values = (10, "Geert", "Vanderkelen")
    add_employee(cursor, 'employee', *values)

    # Make sure data is committed to the database
    cnx.commit()
    cnx.close()

In [ ]:
# add_employees(cursor, employees)
def add_employees(cursor, employees):
    query = "INSERT INTO people(emp_no,first_name, last_name) " \
            "VALUES(%s,%s,%s)"
    for e in employees:
        exec_query(cursor, query, e)
    
if __name__ == '__main__':
    # Connect to database
    cnx = MySQLConnection(**config)
    cursor = cnx.cursor()

    # Get database
    get_database(cnx, 'new_db')

    # Insert new employee
    new_employees = [(10, "Geert", "Vanderkelen"), (11, "Olivier", "Cervello"), (12, "Gertrude", "Roger")]
    add_employees(cursor, new_employees)
    # delete_database(cursor, 'new_db')
                  
    # Make sure data is committed to the database
    cnx.commit()
    cnx.close()

## OOP Approach

In [ ]:
# OOP approach
class Controller(object):
    def __init__(self, cnx):
        self.name = name
        self.cnx = cnx
        self.cursor = cnx.cursor()
        self.tables = []
        
    def get(self, name):
        # Get an existing database named db_name. 
        # If it doesn't exist, create a new database db_name.
        # Also get all tables in database.
        cursor = cnx.cursor()
        print "[INFO]  Fetching database %s ..." % name
        try:
            cnx.database = self.name    
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_BAD_DB_ERROR:
                print "[ERROR] Database %s does not exist! Creating database..." % name
                create_database(cursor, name)
                cnx.database = name
            else:
                print(err)
                exit(1)
        print "[INFO]  Database %s selected." % name

    def create(self, name):
        # Create a new database on the MySQL server
        query = "CREATE DATABASE %s" % self.name
        self.exec_query(query)
        
    def delete(self, name):
        # Delete an existing database.
        query = "DROP DATABASE %s" % name
        self.exec_query(query)
    
    def delete_table(self, name):
        # Delete a table from the database
        query = "DROP TABLE %s" % name
        self.exec_query(query)
            
    def create_table(self, name, args):
        # Create a new table in the database.
        args_str = ','.join(args)
        query = "CREATE TABLE IF NOT EXISTS %s" % name + "(" + args_str + ")"
        self.exec_query(query)
    
    def describe_table(self, name):
        query = "DESCRIBE TABLE %s" % name
        self.exec_query(query)
        
    def exec_query(self, query, query_args=None):
        query_str = query
        try:
            if query_args is not None:
                query = query % query_args
                print "[INFO]  Executing SQL: \"%s\"..." % query,
                self.cursor.execute(query, query_args)
            else:
                print "[INFO]  Executing SQL: \"%s\"..." % query,
                self.cursor.execute(query)
            print " SUCCESS"
        except mysql.connector.Error as err:
            print
            print "[ERROR] " + err.msg
            exit(1)
        
if __name__ == '__main__':
    cnx = mysql.connector.connect(user='root', password='password')
    controller = Controller(cnx)
    controller.get('new_db')
    controller.create_table('new_table', ["hey varchar(10) NOT NULL", "bro varchar(15) NOT NULL", "lol varchar(10) NOT NULL"])
    controller.delete_table('new_table')
    controller.delete('new_db')

# NoSQL Data Access (DynamoDB)

In [ ]:
import boto3

### Client
**Description:** A low-level client representing Amazon DynamoDB.

** Methods available **

** Tables **
* ```list_tables()```
* ```create_table()```
* ```update_table```
* ```delete_table()```
* ```describe_table()```

** Items **
* ```get_item()```
* ```batch_get_item()```
* ```put_item()```
* ```batch_write_item()```
* ```delete_item()```

** Queries **
* ```query()```
* ```scan()```

### Resource
**Description:** A resource representing Amazon DynamoDB.

**Methods available**
* ```batch_get_item()```
* ```batch_write_item()```
* ```create_table()```

** Sub-resources **
* ```Table()``` - a Table resource

** Collections **
* ```tables``` - a collection of Table resources

### ```Table```
**Description:**
A ```Table``` resource.

**Parameters**
* ```name(string)``` - The table's name identifier. This **must** be set.

**Attributes**
* ```attribute_definitions```
* ```creation_date_time```
* ```global_secondary_indexes```
* ```item_count```
* ```key_schema```
* ```latest_stream_arn```
* ```local_secondary_indexes```
* ```provisioned_throughput```
* ```stream_specification```
* ```table_arn```
* ```table_name```
* ```table_size_bytes```
* ```table_status```

**Methods available**
* ```get_item()```
* ```put_item()```
* ```batch_writer()```
* ```delete()```
* ```delete_item()```
* ```query()```
* ```reload()```
* ```scan()```
* ```update()```
* ```update_item()```


### ```tables```
**Description:** A collection of ```Table``` resources.

**Methods**
* ```all()``` - iterable of Table resources in the collection
* ```filter(**kwargs)``` - iterable of Table resources in the collection **filtered by ```kwargs```**
* ```limit(**kwargs)``` - iterable **up to a specified amount** of Table resources in the collection
* ```page_size(**kwargs)``` - iterable of all Table resources in the collection, but **limits the number of items returned** by each service call by the specified amount.

## Connecting to DynamoDB

In [ ]:
dynamodb_client = boto3.client('dynamodb', endpoint_url="http://localhost:9090")
dynamodb_client

In [ ]:
dynamodb_resource = boto3.resource('dynamodb', endpoint_url="http://localhost:9090")
dynamodb_resource

## Creating / Deleting a table

**Client**

In [ ]:
from botocore.exceptions import ClientError

try:
    table = dynamodb_client.create_table(
        TableName='Movies',
        KeySchema=[
            {
                'AttributeName': 'year',
                'KeyType': 'HASH'  #Partition key
            },
            {
                'AttributeName': 'title',
                'KeyType': 'RANGE'  #Sort key
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'year',
                'AttributeType': 'N'
            },
            {
                'AttributeName': 'title',
                'AttributeType': 'S'
            },

        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 10,
            'WriteCapacityUnits': 10
        }
    )
except ClientError as e:
    print "Table already exists."
    
dynamodb_client.delete_table(TableName='Movies')
dynamodb_client.list_tables()

**Resource**

In [ ]:
from botocore.exceptions import ClientError

try:
    table = dynamodb_resource.create_table(
        TableName='Movies',
        KeySchema=[
            {
                'AttributeName': 'year',
                'KeyType': 'HASH'  #Partition key
            },
            {
                'AttributeName': 'title',
                'KeyType': 'RANGE'  #Sort key
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'year',
                'AttributeType': 'N'
            },
            {
                'AttributeName': 'title',
                'AttributeType': 'S'
            },

        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 10,
            'WriteCapacityUnits': 10
        }
    )
except ClientError as e:
    print "Table already exists."

movies = dynamodb_resource.Table('Movies')
movies
# movies.delete()

## Populating a table

**Load JSON Data**

In [ ]:
import json
import decimal

with open("files/moviedata.json") as json_file:
    movies = json.load(json_file, parse_float = decimal.Decimal)
    for movie in movies:
        year = movie['year']
        title = movie['title']
        info = movie['info']
        print "Adding movie:", year, title
        response = table.put_item(
            Item={
                'year': year,
                'title':title,
                'info': info,
            }
        )

**Add item**

In [ ]:
from __future__ import print_function # Python 2/3 compatibility
import json
import decimal

# Helper class to convert a DynamoDB item to JSON.
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            if o % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)

dynamodb = boto3.resource('dynamodb',endpoint_url="http://localhost:9090")

table = dynamodb.Table('Movies')

title = "The Big New Movie"
year = 2015

response = table.put_item( # will overwrite any item with the same keys.
   Item={
        'year': year,
        'title': title,
        'info': {
            'plot':'Something happens.'
        }
    }
)

print("PutItem succeeded:")
print(json.dumps(response, indent=4, cls=DecimalEncoder))

**Add item with condition**

In [ ]:
from __future__ import print_function # Python 2/3 compatibility
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

try:
    table.put_item(
        Item={
            'year': year,
            'title': title,
            'info':{
                'plot':"Nothing happens at all.",
                'rating': decimal.Decimal(0)
            }
        },
        ConditionExpression=Attr("year").ne(year) & Attr("title").ne(title)
    )
except ClientError as e:
    if e.response['Error']['Code'] == "ConditionalCheckFailedException":
        print(e.response['Error']['Message'])
    else:
        raise
else:
    print("PutItem succeeded:")

response = table.get_item(
    Key={
        'year': year,
        'title': title
    }
)
item = response['Item']

**Update an item**

In [ ]:
table = dynamodb.Table('Movies')

title = "The Big New Movie"
year = 2015

response = table.update_item(
    Key={
        'year': year,
        'title': title
    },
    UpdateExpression="set info.rating = :r, info.plot=:p, info.actors=:a",
    ExpressionAttributeValues={
        ':r': decimal.Decimal(5.5),
        ':p': "Everything happens all at once.",
        ':a': ["Larry", "Moe", "Curly"]
    },
    ReturnValues="UPDATED_NEW"
)

print("PutItem succeeded:")
print(json.dumps(response, indent=4, cls=DecimalEncoder))

**Update an item with condition**

In [ ]:
table = dynamodb.Table('Movies')

title = "The Big New Movie"
year = 2015

# Conditional update (will fail)
print("Attempting conditional update...")

try:
    response = table.update_item(
        Key={
            'year': year,
            'title': title
        },
        UpdateExpression="remove info.actors[0]",
        ConditionExpression="size(info.actors) >= :num",
        ExpressionAttributeValues={
            ':num': 3
        },
        ReturnValues="UPDATED_NEW"
    )
except ClientError as e:
    if e.response['Error']['Code'] == "ConditionalCheckFailedException":
        print(e.response['Error']['Message'])
    else:
        raise
else:
    print("PutItem succeeded:")

print(json.dumps(response, indent=4, cls=DecimalEncoder))

**Delete an item**

In [ ]:
table = dynamodb.Table('Movies')

title = "The Big New Movie"
year = 2015

print("Attempting a conditional delete...")

try:
    response = table.delete_item(
        Key={
            'year': year,
            'title': title
        },
        ConditionExpression="info.rating <= :val",
        ExpressionAttributeValues= {
            ":val": decimal.Decimal(5)
        }
    )
except ClientError as e:
    if e.response['Error']['Code'] == "ConditionalCheckFailedException":
        print(e.response['Error']['Message'])
    else:
        raise
else:
    print("PutItem succeeded:")

print(json.dumps(response, indent=4, cls=DecimalEncoder))

## Querying data from a table

**Query: All movies released in a year**

In [ ]:
table = dynamodb.Table('Movies')

print("Movies from 1985")

response = table.query(
    KeyConditionExpression=Key('year').eq(1985)
)

for i in response['Items']:
    print(i['year'], ":", i['title'])

**Query: All movies released in a year with certain titles**

In [ ]:
table = dynamodb.Table('Movies')

print("Movies from 1992 - titles A-L, with genres and lead actor")

response = table.query(
    ProjectionExpression="#yr, title, info.genres, info.actors[0]",
    ExpressionAttributeNames={ "#yr": "year" }, # Expression Attribute Names for Projection Expression only.
    KeyConditionExpression=Key('year').eq(1985) & Key('title').between('A', 'L')
)

for i in response[u'Items']:
    print(json.dumps(i, cls=DecimalEncoder))

**Scan**

In [ ]:
table = dynamodb.Table('Movies')

fe = Key('year').between(1950, 1959);
pe = "#yr, title, info.rating"
# Expression Attribute Names for Projection Expression only.
ean = { "#yr": "year", }
esk = None


response = table.scan(
    FilterExpression=fe,
    ProjectionExpression=pe,
    ExpressionAttributeNames=ean
    )

for i in response['Items']:
    print(json.dumps(i, cls=DecimalEncoder))

while 'LastEvaluatedKey' in response:
    response = table.scan(
        ProjectionExpression=pe,
        FilterExpression=fe,
        ExpressionAttributeNames= ean,
        ExclusiveStartKey=response['LastEvaluatedKey']
        )

    for i in response['Items']:
        print(json.dumps(i, cls=DecimalEncoder))

# Exercise: Parse a switch record and output to MySQL / DynamoDB

In [9]:
import csv